In [309]:
from openstack import connection
from random import randint
import copy
from collections import Counter
from openpyxl import *
import os
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import colors, Alignment
from openpyxl.cell import Cell

In [2]:
def connection_to_cluster(project):
    conn = connection.Connection(auth_url="http://controller:35357/v3",
                                 project_name=project,
                                 project_domain_name="default",
                                 user_domain_name="default",
                                 username="admin",
                                 password="QIP@t4Ay")
    return conn

def get_servers(conn):
    a = [i for i in conn.compute.servers(all_tenants=True)]
    return a

def create_real_flavor_dict(maxram=1000000):
    
    fl = [[fId, conn.compute.get_flavor(flavor=fId).vcpus, conn.compute.get_flavor(flavor=fId).ram]\
               for fId in set([y.flavor['id'] for y in servers_list if y.flavor['id'] != '1']) \
               if conn.compute.get_flavor(flavor=fId).ram < maxram]
    flu = [list(x) for x in set(tuple(x) for x in fl)]
    fd ={}
    for i in range(0,len(flu)):
        fd[flu[i][0]]=[flu[i][1],flu[i][2]]
        
        
    return fd

In [3]:
#####  
def create_test_data(fdict,gnum,maxg,ming):
    fdict=dict(fdict)
    ng = 1
    mn = 1
    returnDict_groups_vm = {}
    returnDict_name_vm   = {}
    while ng < gnum+1:
        returnDict_groups_vm[ng]={}
        vmn=randint(ming,maxg)
        for i in range(1,vmn+1):
            res = list(fdict[randint(1,len(fdict.keys()))])
            res.append(mn)
            returnDict_groups_vm[ng][i]= res
            returnDict_name_vm[mn] = {}
            returnDict_name_vm[mn]['name'] = "gr"+str(ng)+"-name-"+str(i)
            returnDict_name_vm[mn]['vcpu']=res[0]
            returnDict_name_vm[mn]['ram']=res[1]
            returnDict_name_vm[mn]['groupid']=ng
            mn=mn+1
        ng = ng + 1
    
    return returnDict_groups_vm, returnDict_name_vm, mn-1

    
def create_flavor_dict(maxram=1000000):
    fl = [[conn.compute.get_flavor(flavor=fId).vcpus, conn.compute.get_flavor(flavor=fId).ram]\
               for fId in set([y.flavor['id'] for y in servers_list if y.flavor['id'] != '1']) \
               if conn.compute.get_flavor(flavor=fId).ram < maxram]
    flu = [list(x) for x in set(tuple(x) for x in fl)]
    fd ={}
    for i in range(0,len(flu)):
        fd[i+1]=flu[i]
    return fd

    


#####


def create_hypervisor_dict(noz=3,noh=10,hcpu=1,hram=1024):
    rd={}
    hprefix='CMP'
    min_h_in_az = noh // noz
    hnumber=1
    aznumber=1
    while aznumber < noz + 1:
        
        #print(hnumber * aznumber)
        if hnumber < min_h_in_az * aznumber:
            rd[hnumber]={'name':hprefix+str(hnumber),\
                         'az':aznumber,\
                         'free_cpu':hcpu,\
                         'free_ram':hram, \
                         'perc_ram' : 1,\
                         'perc_cpu' : 1,\
                         'vm_count' : 0}
            hnumber = hnumber + 1
            #print('1',hnumber,aznumber)
        elif hnumber > noh:
            #print('end',hnumber)
            break
        elif aznumber == noz:
            rd[hnumber]={'name':hprefix+str(hnumber),\
                         'az':aznumber,\
                         'free_cpu':hcpu,\
                         'free_ram':hram,\
                         'perc_ram' : 1,\
                         'perc_cpu' : 1,\
                         'vm_count' : 0}
            hnumber = hnumber + 1
            #print('2',hnumber,aznumber)
        else:
            aznumber = aznumber + 1
    return rd, copy.deepcopy(rd)


#######


In [4]:
%%time
## ask OpenStack about servers and flavors
conn = connection_to_cluster('admin')
servers_list = get_servers(conn)
#flavor_dict = create_flavor_dict()
flavor_dict = create_real_flavor_dict()

CPU times: user 1.74 s, sys: 85.6 ms, total: 1.83 s
Wall time: 21.9 s


In [172]:

######



def get_groups_from_vms(sl):
    rd_vm={}
    rd_grp={}
    not_allocate=[]
    vm_id = 0
    group_id = 0
    l_groups = [''.join(y for y in i.name if not y.isdigit()) for i in sl\
                if ''.join(y for y in i.name if not y.isdigit()) != ''] 
    
    ### группа для машины 1007
    l_groups.append(str('1007'))
    #print(l_groups)
    l_with_names = [[i.name, i.hypervisor_hostname] for i in sl]
    for i in sl:
        #print(rd_vm)
        if i.name in kill_list:
            continue
        if i.flavor['id'] == '1':
            not_allocate.append(i.name)
            #print(i.name)
            continue
        if 'bragin' in i.name:
            not_allocate.append(i.name)
            #print(i.name)
            continue
        if 'zbx' in i.name:
            not_allocate.append(i.name)
            #print(i.name)
            continue   
        if 'shelved' in i.name:
            not_allocate.append(i.name)
            #print(i.name)
            continue
        if i.name in cr1_list:
            cr = 2
        elif i.name in cr2_list:
            cr = 1
        else:
            cr = 0
        vm_id = vm_id + 1
        rd_vm[vm_id] = {}
        rd_vm[vm_id]['name'] = i.name
        rd_vm[vm_id]['vcpu'] = flavor_dict[i.flavor['id']][0]
        rd_vm[vm_id]['ram']  = flavor_dict[i.flavor['id']][1]
        rd_vm[vm_id]['crit_rate'] = cr
        #rd_vm[vm_id]['id']  = vm_id
    #print(set(l_groups))    
    for i in set(l_groups):
        #print(i)
        group_id = group_id + 1 
        rd_grp[group_id] = {}
        rd_grp[group_id]['_name'] = i
        k = 0
        for srv in rd_vm.keys():
            if ''.join(y for y in rd_vm[srv]['name'] if not y.isdigit()) == i:
                k= k + 1
                rd_grp[group_id][k]=[rd_vm[srv]['vcpu'], rd_vm[srv]['ram'], rd_vm[srv]['name'], srv]
                rd_vm[srv]['groupid'] = group_id
            elif rd_vm[srv]['name'] == '1007':
                #print(i,rd_vm[srv]['name'])
                k= k + 1
                rd_grp[group_id][k]=[rd_vm[srv]['vcpu'], rd_vm[srv]['ram'], rd_vm[srv]['name'], srv]
                rd_vm[srv]['groupid'] = group_id 
            #print(rd_vm[srv]['name'],i)
    return rd_grp, rd_vm, vm_id, not_allocate



def create_group_in_use_dict(gdict,noz=3):
    rd={}
    for i in gDict.keys():
        rd[i]={}
        rd[i]['vmtotal']=len(gDict[i].keys())
        rd[i]['az']={}
        rd[i]['hyp']={}
        for az in range(1,noz+1):
            rd[i]['az'][az]=0
        for h in range(1,number_of_hyp+1):
            rd[i]['hyp'][h]=0
    return rd, copy.deepcopy(rd)

#######

def count_over (h,v):
    h=dict(h)
    v=dict(v)
    vm_vcpu_sum = sum([v[i]['vcpu'] for i in v.keys()])
    vm_ram_sum = sum([v[i]['ram'] for i in v.keys()])
    hyper_vcpu_sum = sum([h[i]['free_cpu'] for i in h.keys()])
    hyper_ram_sum = sum([h[i]['free_ram'] for i in h.keys()])
    
    return [vm_vcpu_sum, vm_ram_sum], [hyper_vcpu_sum,hyper_ram_sum], [vm_vcpu_sum / hyper_vcpu_sum, vm_ram_sum/hyper_ram_sum] 


#######


def calculate_after_loc(v,h,hdictinuse,gdictinuse):
    ### Изменения во временных словарях после локации
    hdictinuse[h]['free_cpu'] = (hdictinuse[h]['free_cpu'] - vDict[v]['vcpu']) 
    hdictinuse[h]['perc_cpu'] = (hdictinuse[h]['free_cpu'] - vDict[v]['vcpu']) / hyper_cpu
    hdictinuse[h]['free_ram'] = (hdictinuse[h]['free_ram'] - vDict[v]['ram']) 
    hdictinuse[h]['perc_ram'] = (hdictinuse[h]['free_ram'] - vDict[v]['ram']) / hyper_ram
    hdictinuse[h]['vm_count'] = (hdictinuse[h]['vm_count'] + 1)
    gdictinuse[vDict[v]['groupid']]['az'][hDict[h]['az']] = gdictinuse[vDict[v]['groupid']]['az'][hDict[h]['az']] + 1 
    gdictinuse[vDict[v]['groupid']]['hyp'][h] = gdictinuse[vDict[v]['groupid']]['hyp'][h] + 1
    hdictinuse[h]['vms_ids'].append(v)
    if vDict[v]['crit_rate'] == 2:
        hdictinuse[h]['vm_critical'] = hdictinuse[h]['vm_critical'] + 1
    if vDict[v]['crit_rate'] == 1:
        hdictinuse[h]['vm_warning'] = hdictinuse[h]['vm_warning'] + 1
        

    
    
def get_hid_by_score(hd,gd,vid):
    rd={}
    #print(vDict[vid]['name'])
    for h in hDict.keys():
       
        group = vDict[vid]['groupid']
        az = hDict[h]['az']
        vmt = gd[group]['vmtotal']
        machine_in_az = gd[group]['az'][az]
        machine_in_hyp = gd[group]['hyp'][h]
        #critical_in_hyp = len()
        #print((hd[h]['free_ram'] - vDict[vid]['ram']) / hyper_ram)
        machine_warning_on_hyp = float(hd[h]['vm_warning'])
        if vDict[vid]['crit_rate'] == 2 and hd[h]['vm_critical'] > 0 :
            score=-1000
        elif (hd[h]['free_ram'] - vDict[vid]['ram']) / hyper_ram < 0.01:
            score=-5
        elif (hd[h]['perc_cpu'] < 0):
            score = count_score(0,hd[h]['perc_ram'], machine_in_az, machine_in_hyp,vmt,machine_warning_on_hyp)
        else:  
            score = count_score(hd[h]['perc_cpu'],hd[h]['perc_ram'], machine_in_az, machine_in_hyp,vmt,machine_warning_on_hyp)
        rd[h]={#'mply': score_mply,\
                          # 'score_az':score_az,\
                          # 'score_hyp':score_hyp, 
                           'score': score,\
                           'id' : h,\
                           'ram': hd[h]['free_ram'],\
                           'cpu': hd[h]['free_cpu'],\
                           'vm_count' : hd[h]['vm_count']
                          }
    ll = [[rd[i]['id'], rd[i]['score']] for i in rd.keys()]
    winner_list = [i for i in range(0, len(sorted(ll, key = lambda x: float(x[1]), reverse=True))) if \
    sorted(ll, key = lambda x: float(x[1]), reverse=True)[0][1] ==\
    sorted(ll, key = lambda x: float(x[1]), reverse=True)[i][1]]
    #print(winner_list)
    winner = sorted(ll, key = lambda x: float(x[1]), reverse=True)[randint(0,len(winner_list)-1)]
    ## winner[0] - hid, winnder[1] - score
    return winner[0], winner[1]



def count_score(pc,pr,ma,mh,vmt,mw):
    if mw == 0:
        mw == 0
    else:
        mw = mw * 0.1 + 0.1
    res = (0.5 * pc + 2 * pr)/2
    az  = (1 - ma / vmt)
    hz  = (1 - mh / vmt)

    score = 1.3 * res * 1.2 * az * 1.3 * hz * (1 - mw)
    #print(mw)
    return score


def create_real_hypervisors(al):
    hid = 0 
    rd={}
    for i in al:
        if i[1] in m_list:
            continue
        hid = hid + 1
        rd[hid]={
                         'name':i[0].replace('.egron.tnx',''),\
                         'label':i[1],\
                         'az':int(i[2]),\
                         'free_cpu':hyper_cpu,\
                         'free_ram':hyper_ram, \
                         'perc_ram' : 1,\
                         'perc_cpu' : 1,\
                         'vm_count' : 0,\
                         'vms_ids': [],
                         'vm_critical': 0,\
                         'vm_warning' : 0}
    return rd, copy.deepcopy(rd)
        



In [173]:
#x, y, z, e = get_groups_from_vms(servers_list)


In [174]:
#z, len(e), len(servers_list), e

In [175]:
with open('./az1', 'r') as f:
    az1_file = f.readlines()
    az1_file = [i.replace('\n','').split(' ') for i in az1_file]
f.close
with open('./az2', 'r') as f:
    az2_file = f.readlines()
    az2_file = [i.replace('\n','').split(' ') for i in az2_file]
f.close
with open('./az3', 'r') as f:
    az3_file = f.readlines()
    az3_file = [i.replace('\n','').split(' ') for i in az3_file]
f.close

with open('./cr1', 'r') as f:
    cr1_list = f.readlines()
    cr1_list = [i.replace('\n','') for i in cr1_list]
f.close
with open('./cr2', 'r') as f:
    cr2_list = f.readlines()
    cr2_list = [i.replace('\n','') for i in cr2_list]
f.close
with open('./cr2', 'r') as f:
    cr2_list = f.readlines()
    cr2_list = [i.replace('\n','') for i in cr2_list]
f.close
with open('./migr_nodes', 'r') as f:
    m_list = f.readlines()
    m_list = [i.replace('\n','') for i in m_list]
f.close

with open('./out_1', 'r') as f:
    kill_list = f.readlines()
    kill_list = [i.replace('\n','') for i in kill_list]
f.close


_az_list = az1_file + az2_file + az3_file


In [176]:
## fake variables
number_of_groups = 200
max_vm_in_group = 15
min_vm_in_group = 2
#number_of_hyp = 76

hyper_cpu = 56
hyper_ram = 512000


In [177]:
len(az1_file), len(az2_file), len(az3_file)

(41, 16, 18)

In [178]:
#for i in gDict.keys():
#    if len(gDict[i]) > 60: 
#        print(len(gDict[i]), gDict[i]['_name'])

In [179]:
## create test data - dcit of groups and dict of vms
#gDict, vDict, vm_count = create_test_data(flavor_dict,number_of_groups,max_vm_in_group,min_vm_in_group)
#gDict, vDict, vm_count = create_test_data(flavor_dict,number_of_groups,max_vm_in_group,min_vm_in_group)
gDict, vDict, vm_count, not_allocated = get_groups_from_vms(servers_list)
#hDict, hDict_in_use = create_hypervisor_dict(noz=3,noh=number_of_hyp,hram=hyper_ram,hcpu=hyper_cpu)
hDict, hDict_in_use = create_real_hypervisors(_az_list)
number_of_hyp = len(hDict.keys())
gDictLocation, gDictLocation_in_use = create_group_in_use_dict(gDict,noz=3)
vm_res_total_list, hyper_res_total_list, over_total_list = count_over(hDict,vDict)

In [180]:
#hDict

In [181]:
#[[vDict[i]['crit_rate'], vDict[i]['name']] for i in vDict.keys() if vDict[i]['crit_rate'] == 1]

In [182]:
not_allocated

['test_3par_attach', 'test_shelved', 'zbx_bcp']

In [183]:
#vm_count

In [184]:
print("Всего машин:", vm_count)
print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
print("На одном ВУ CPU :", hyper_cpu, ", RAM :", hyper_ram, "MB")
print("Всего ВУ:", number_of_hyp)
print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

Всего машин: 1521
Необходимо машинам vCPU : 12294 , RAM : 33618360 MB
На одном ВУ CPU : 56 , RAM : 512000 MB
Всего ВУ: 65
Суммарно на ВУ : 3640 , RAM : 33280000 MB
Суммарная подписка по CPU : 3.3774725274725275 , RAM : 1.0101670673076923


In [185]:
%%time
scoring_list=[]
iteration=0
for r,i in sorted([[vDict[i]['crit_rate'],i] for i in vDict.keys()], key = lambda x: int(x[0]),reverse=True):
    iteration=iteration + 1 
    hid, hid_score = get_hid_by_score(gd=gDictLocation_in_use,hd=hDict_in_use,vid=i)
    calculate_after_loc(gdictinuse=gDictLocation_in_use,hdictinuse=hDict_in_use,h=hid,v=i)
    scoring_list.append([iteration, i, r, hid, hid_score])
    #print(i, r, hid, hid_score)

CPU times: user 4.67 s, sys: 0 ns, total: 4.67 s
Wall time: 4.67 s


In [186]:
s = [h[4] for h in scoring_list]
print("Значения score: ")
print("Максимальный скоринг на итерации :", max(s))
print("Минимальный скоринг на итерации :", min(s))
print("Средний :", sum(s) / len(s) )


Значения score: 
Максимальный скоринг на итерации : 2.535
Минимальный скоринг на итерации : -5
Средний : 0.3867639808274227


In [187]:
l = [hDict_in_use[h]['perc_ram'] for h in hDict.keys()]
m = [hDict_in_use[h]['perc_cpu'] for h in hDict.keys()]

print("Забитость ВУ по RAM после распределения ")
print("Самый забитый :", 1-min(l))
print("Наименее забитый :", 1-max(l))
print("Средняя загруженность :",  1 - sum(l)/len(l))
print('')
print("Перерасход vCPU относительно CPU ВУ после распределения ")
print("Самый забитый :", 1-min(m))
print("Наименее забитый :", 1-max(m))
print("Средняя загруженность :", 1 - sum(m)/len(m))

Забитость ВУ по RAM после распределения 
Самый забитый : 1.230125
Наименее забитый : 0.976
Средняя загруженность : 1.0511516826923077

Перерасход vCPU относительно CPU ВУ после распределения 
Самый забитый : 4.25
Наименее забитый : 2.7857142857142856
Средняя загруженность : 3.5362637362637352


In [193]:
for i in gDict.keys():
    for j in gDictLocation_in_use[i]['hyp'].keys():
        if gDictLocation_in_use[i]['hyp'][j] > 2:
            print(hDict[j]['label'], gDict[i]['_name'], gDictLocation_in_use[i]['hyp'][j], len(gDict[i].keys()))

S_CMP48 isurz-uploader- 3 41
S_CMP70 isurz-uploader- 3 41
S_CMP58 pg-attribute-standby- 3 94
S_CMP10 pg-attribute-standby- 3 94
S_CMP16 pg-attribute-standby- 3 94
S_CMP24 pg-attribute-standby- 3 94
S_CMP31 isurz-xml-storage- 3 93
S_CMP36 isurz-xml-storage- 3 93
S_CMP43 isurz-xml-storage- 3 93
S_CMP50 isurz-xml-storage- 3 93
S_CMP53 isurz-xml-storage- 3 93
S_CMP58 isurz-xml-storage- 3 93
S_CMP06 isurz-xml-storage- 3 93
S_CMP16 isurz-xml-storage- 3 93
S_CMP27 isurz-xml-storage- 3 93
S_CMP28 isurz-xml-storage- 4 93
S_CMP29 isurz-xml-storage- 3 93
S_CMP40 migr-gkn-oracle- 3 92
S_CMP53 migr-gkn-oracle- 3 92
S_CMP61 migr-gkn-oracle- 3 92
S_CMP70 migr-gkn-oracle- 3 92
S_CMP10 migr-gkn-oracle- 3 92
S_CMPZ01 migr-gkn-oracle- 3 92
S_CMPZ02 migr-gkn-oracle- 3 92
S_CMP32 migr-node- 3 81
S_CMP41 migr-node- 3 81
S_CMP46 migr-node- 3 81
S_CMP62 migr-node- 3 81
S_CMP67 migr-node- 3 81
S_CMP15 migr-node- 3 81
S_CMP22 migr-node- 3 81
S_CMP48 migr-grp-oracle- 4 61
S_CMP50 migr-grp-oracle- 3 61
S_CMP70 mi

In [194]:
hDict

{1: {'az': 1,
  'free_cpu': 56,
  'free_ram': 512000,
  'label': 'S_CMP31',
  'name': 'cn-e612',
  'perc_cpu': 1,
  'perc_ram': 1,
  'vm_count': 0,
  'vm_critical': 0,
  'vm_warning': 0,
  'vms_ids': []},
 2: {'az': 1,
  'free_cpu': 56,
  'free_ram': 512000,
  'label': 'S_CMP32',
  'name': 'cn-d724',
  'perc_cpu': 1,
  'perc_ram': 1,
  'vm_count': 0,
  'vm_critical': 0,
  'vm_warning': 0,
  'vms_ids': []},
 3: {'az': 1,
  'free_cpu': 56,
  'free_ram': 512000,
  'label': 'S_CMP33',
  'name': 'cn-e80e',
  'perc_cpu': 1,
  'perc_ram': 1,
  'vm_count': 0,
  'vm_critical': 0,
  'vm_warning': 0,
  'vms_ids': []},
 4: {'az': 1,
  'free_cpu': 56,
  'free_ram': 512000,
  'label': 'S_CMP34',
  'name': 'cn-e814',
  'perc_cpu': 1,
  'perc_ram': 1,
  'vm_count': 0,
  'vm_critical': 0,
  'vm_warning': 0,
  'vms_ids': []},
 5: {'az': 1,
  'free_cpu': 56,
  'free_ram': 512000,
  'label': 'S_CMP35',
  'name': 'cn-d720',
  'perc_cpu': 1,
  'perc_ram': 1,
  'vm_count': 0,
  'vm_critical': 0,
  'vm_warnin

In [171]:
[[hDict_in_use[i]['vm_count'], hDict_in_use[i]['vm_critical'], hDict_in_use[i]['vm_warning']] for i in hDict.keys()]

[[25, 1, 3],
 [25, 0, 4],
 [22, 0, 3],
 [20, 1, 4],
 [21, 0, 3],
 [22, 0, 4],
 [21, 1, 3],
 [28, 0, 3],
 [22, 1, 3],
 [25, 1, 3],
 [26, 1, 3],
 [23, 1, 3],
 [25, 1, 3],
 [22, 0, 3],
 [24, 1, 3],
 [27, 1, 3],
 [27, 1, 3],
 [22, 1, 3],
 [23, 1, 3],
 [21, 0, 3],
 [18, 0, 3],
 [24, 0, 3],
 [20, 1, 3],
 [29, 0, 3],
 [24, 1, 3],
 [31, 0, 3],
 [22, 1, 3],
 [25, 1, 3],
 [27, 1, 3],
 [26, 0, 3],
 [20, 0, 4],
 [24, 1, 3],
 [23, 1, 4],
 [22, 0, 3],
 [23, 0, 3],
 [28, 0, 3],
 [23, 0, 5],
 [23, 1, 4],
 [23, 1, 4],
 [18, 1, 4],
 [25, 1, 4],
 [25, 1, 4],
 [24, 1, 4],
 [18, 1, 4],
 [21, 1, 4],
 [25, 1, 4],
 [23, 0, 5],
 [15, 1, 3],
 [27, 0, 4],
 [23, 0, 4],
 [17, 1, 4],
 [22, 0, 4],
 [27, 1, 4],
 [24, 0, 4],
 [24, 0, 4],
 [22, 1, 4],
 [19, 0, 4],
 [24, 0, 4],
 [24, 0, 4],
 [27, 0, 4],
 [23, 1, 4],
 [21, 0, 4],
 [30, 0, 5],
 [21, 1, 4],
 [26, 0, 5]]

In [ ]:
def send_this_to_excel(hd):
    print(i)

In [219]:
vDict[1]

{'crit_rate': 0,
 'groupid': 166,
 'name': 'pkurp-app-040',
 'ram': 8192,
 'vcpu': 4}

In [197]:
hDict_in_use[1]

{'az': 1,
 'free_cpu': -136,
 'free_ram': 20480,
 'label': 'S_CMP31',
 'name': 'cn-e612',
 'perc_cpu': -2.5714285714285716,
 'perc_ram': 0.008,
 'vm_count': 23,
 'vm_critical': 0,
 'vm_warning': 4,
 'vms_ids': [161,
  393,
  916,
  1272,
  524,
  529,
  532,
  533,
  539,
  540,
  579,
  755,
  759,
  942,
  944,
  1020,
  1111,
  1116,
  1170,
  1227,
  1321,
  1408,
  1411]}

In [406]:
file="./1.xlsx"
wb = Workbook ()
try:
    os.remove(file)
except OSError:
    pass
ws = wb.active
ws.title=('Распределение ВМ по ВУ')
warningFill = PatternFill(start_color='ffcc33',
                   end_color='ffcc33', fill_type='solid')
criticalFill = PatternFill(start_color='ee4000',
                   end_color='ee4400', fill_type='solid')
headersFill = PatternFill(start_color='ebecec',
                   end_color='ebecec', fill_type='solid')
vm_headersFill = PatternFill(start_color='faebd7',
                   end_color='faebd7', fill_type='solid')
top_headersFill = PatternFill(start_color='8c98a0',
                   end_color='8c98a0', fill_type='solid')

row = 3
col = 1
t=hDict_in_use
ws.merge_cells('A1:E1')

for i in t.keys():
    for rows in ws.iter_rows(min_row=row - 1, max_row=row - 1 , min_col=1, max_col=7):
        for cell in rows:
            cell.font = Font(italic=True)
            cell.alignment= Alignment(horizontal='center')
            cell.fill = top_headersFill
    ws.cell(column = col, row = row - 1 , value='Имя ВУ').alignment= Alignment(horizontal='left')
    ws.cell(column = col + 1, row = row  - 1, value='Номер зоны доступности')
    ws.cell(column = col + 2, row = row - 1, value='Переподписка по CPU')
    ws.cell(column = col + 3, row = row - 1, value='Переподписка по RAM')
    ws.cell(column = col + 4, row = row - 1, value='Количество ВМ на ВУ')
    ws.cell(column = col + 5, row = row - 1, value='Количество ВМ первого уровня критичности')
    ws.cell(column = col + 6, row = row - 1, value='Количество ВМ второго уровня критичности')
    for rows in ws.iter_rows(min_row=row + 1, max_row=row + 1 , min_col=1, max_col=7):
        for cell in rows:
            cell.font = Font(italic=True) 
            cell.alignment= Alignment(horizontal='right')
    ws.cell(column=col, row=row).font = Font(name='Calibri', bold=True)
    for rows in ws.iter_rows(min_row=row , max_row=row, min_col=1, max_col=7):
        for cell in rows:
              cell.fill = headersFill 
    ws.cell(column=col, row=row, value=t[i]['label'])
    ws.cell(column=col + 1, row=row, value=t[i]['az'])
    ws.cell(column=col + 2, row=row, value="{:10.2f}".format((1 - t[i]['perc_cpu']) * 100)).alignment = Alignment(horizontal='center')
    ws.cell(column=col + 3, row=row, value="{:10.2f}".format((1 - t[i]['perc_ram']) * 100)).alignment = Alignment(horizontal='center')
    ws.cell(column=col + 4, row=row, value=t[i]['vm_count'])
    ws.cell(column=col + 5, row=row, value=t[i]['vm_critical']) 
    ws.cell(column=col + 6, row=row, value=t[i]['vm_warning']) 
    row = row + 1
    for rows in ws.iter_rows(min_row=row, max_row=row, min_col=1, max_col=7):
        for cell in rows:
            cell.fill = vm_headersFill
            cell.alignment = Alignment(horizontal='center')
    ws.cell(column=col + 2, row=row, value='Имя ВМ').alignment = Alignment(horizontal='left')
    ws.cell(column=col + 3, row=row, value='vCPU')
    ws.cell(column=col + 4, row=row, value='RAM') 
    ws.cell(column=col + 5, row=row, value='Уровень критичности')
    ws.cell(column=col + 6, row=row, value='Имя логической группы') 

    row = row + 1
    for j in t[i]['vms_ids']:
        v = vDict[j]
        ws.cell(column=col + 2, row=row, value=v['name'])
        ws.cell(column=col + 3, row=row, value=v['vcpu'])
        ws.cell(column=col + 4, row=row, value=v['ram']) 
        if v['crit_rate'] == 2:
            crit = 1
        elif v['crit_rate'] == 1:
            crit = 2
        elif v['crit_rate'] == 0:
            crit = 'н/д'
        ws.cell(column=col + 5, row=row, value=crit)
        if v['crit_rate'] == 2:
            for rows in ws.iter_rows(min_row=row, max_row=row, min_col=3, max_col=7):
                for cell in rows:
                      cell.fill = criticalFill
        elif v['crit_rate'] == 1:
            for rows in ws.iter_rows(min_row=row, max_row=row, min_col=3, max_col=7):
                for cell in rows:
                      cell.fill = warningFill
           
        ws.cell(column=col + 6, row=row, value=gDict[v['groupid']]['_name']) 
        row = row + 1
    row = row + 2
        
        
        
data = ws.get_cell_collection()
dict2 = dict()
for i in data:
    key=i.column
    subkey=i.row
    value=i.coordinate
    #print(key,subkey,value)
    if key in dict2:
        dict2[key][subkey] = value
    else:
        dict2[key] = {}
        dict2[key][subkey] = value
#print(dict2)
for col in dict2:
    data1=[]
    for cell in dict2[col]:
        data1.append(dict2[col][cell])
    column_widths = []
    for cell in data1:
        cellvalue=ws.cell(cell).value
        #print(cellvalue)
        column_widths.append(len(str(cellvalue)))
    #print(max(column_widths))
    ws.column_dimensions[col].width=max(column_widths)+5

ws = wb.create_sheet(title='Легенда')    
row = 3
col = 1
ws.merge_cells('A1:L1')
ws['A1'] = 'Все ВМ распределены на условные зоны доступности, характиризующиеся подключением к \
 парам коммутаторов'
ws.merge_cells('A2:c2')
ws['A2'] = 'Состав зон доступности'

ws.cell(column=col,row = row,value='1')
for i in [hDict_in_use[i]['label'] for i in hDict.keys() if hDict_in_use[i]['az'] == 1]:
    row=row+1
    ws.cell(column=col,row=row,value=i)
    
    
row =3   
ws.cell(column=col+1,row = row,value='2')
for i in [hDict_in_use[i]['label'] for i in hDict.keys() if hDict_in_use[i]['az'] == 2]:
    row=row+1
    ws.cell(column=col+1,row=row,value=i)
    
row=3
ws.cell(column=col+2,row=row,value='3')
for i in [hDict_in_use[i]['label'] for i in hDict.keys() if hDict_in_use[i]['az'] == 3]:
    row=row+1
    ws.cell(column=col+2,row=row,value=i)
    
        
data = ws.get_cell_collection()
dict2 = dict()
for i in data:
    key=i.column
    subkey=i.row
    value=i.coordinate
    #print(key,subkey,value)
    if key in dict2:
        dict2[key][subkey] = value
    else:
        dict2[key] = {}
        dict2[key][subkey] = value
#print(dict2)
for col in dict2:
    data1=[]
    for cell in dict2[col]:
        data1.append(dict2[col][cell])
    column_widths = []
    for cell in data1:
        cellvalue=ws.cell(cell).value
        #print(cellvalue)
        column_widths.append(len(str(cellvalue)))
    #print(max(column_widths))
    ws.column_dimensions[col].width=max(column_widths)+5
    
    
    
    
wb.save(file)

/usr/lib/python3.4/site-packages/openpyxl/worksheet/worksheet.py:303: UserWarning: Using a coordinate with ws.cell is deprecated. Use ws[coordinate] instead
  warn("Using a coordinate with ws.cell is deprecated. Use ws[coordinate] instead")


In [388]:
gDict[1]

{'_name': 'pmo-armedit-',
 2: [16, 32768, 'pmo-armedit-03', 577],
 3: [16, 32768, 'pmo-armedit-02', 619],
 4: [16, 32768, 'pmo-armedit-01', 934],
 1: [2, 8192, '1007', 260]}

In [360]:
[hDict_in_use[i]['label'] for i in hDict.keys() if hDict_in_use[i]['az'] == 3]

['S_CMPZ01',
 'S_CMPZ02',
 'S_CMPZ03',
 'S_CMP17',
 'S_CMP20',
 'S_CMP21',
 'S_CMP22',
 'S_CMP24',
 'S_CMP25',
 'S_CMP26',
 'S_CMP27',
 'S_CMP28',
 'S_CMP29',
 'S_CMP30',
 'S_CMP76']